# Pseudotime using spectral embedding

In [ ]:
%matplotlib inline
import numpy as np
import math
import os
from numpy.matlib import repmat
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from scipy.interpolate import UnivariateSpline
from scipy import optimize
from OscopeBootstrap.pseudotime import estimate_pseudotime_using_spectral_embedding, calculate_metrics, \
    plot_latent_space, plot_latent_space_notrue, plot_gene_fits, plot_gene_fits_notrue, plot_correspondence_of_peaktime_and_times

## Define a test sin function for the fitting

In [ ]:
def test_func(x, dist, a,b,p):
    return dist + a * np.sin(p + b*x)

## Load data

In [ ]:
case='D8'#D0,D1,D2,D4,D5,D7,D8
alpha='0.2'
path_to_data = '../Glioblastoma_project/OsconetInput/'
path_to_comm = '../Glioblastoma_project/Results_09/'
fig_root='../Glioblastoma_project/Plots09/'
path_to_fig=f'{fig_root}{case}/'
datafile=f'{path_to_data}filter{case}_{alpha}.csv'
data=pd.read_csv(datafile,header=0, index_col=0)
if not os.path.exists(path_to_fig):
    os.makedirs(path_to_fig)


In [ ]:
data

## load community info

In [ ]:
commDataFile=f'{path_to_comm}{case}_{alpha}Comm.csv'
commReportFile=f'{path_to_comm}Report{case}_{alpha}.csv'
commData = pd.read_csv(commDataFile,header=0, index_col=1)
commData=commData[['CommunityID']]
genelist=commData.index.values
ncomm=commData.max()
commInfo=pd.read_csv(commReportFile,header=0, index_col=1)
print(commReportFile)

In [ ]:
commInfo=commInfo.sort_index()


In [ ]:
#commInfo

In [ ]:
communities=commInfo.index.values
currentCFlag=commInfo.loc[communities==1,'LinFlag']
print(currentCFlag.values)
#selecting  linear clusters:

Linear=commInfo[commInfo.LinFlag==1]
LinearId=Linear.index.values
#selecting  non linear clusters:
print('Linear Communities')
print(LinearId)
NonLinear=commInfo[commInfo.LinFlag==0]
NonLinearId=NonLinear.index.values
print('Non Linear Communities')
print(NonLinearId)

In [ ]:
filename=case+' missing plots'
print(filename)

# Get data for the community of interest - Linear Case

In [ ]:
#Start from the linear communities
print('LINEAR COMMUNITIES PLOT')
filename=f'{path_to_fig}{case}_missing_plots.txt'
if os.path.exists(filename):
    os.remove(filename)
    
file_obj=open(filename,'a')
file_obj.write("LINEAR COMMUNITIES MISSING PLOTS \n")
for commId in LinearId:
    print('Community=')
    print(commId)
    file_obj.write(f'Community ={commId}')
    file_obj.write("\n ")

    #commId=LinearId[4]
    Flag="Linear"
    comm = commData[commData.CommunityID == commId]
    #select ONLY the genes in the selected community to use laterin the plot
    genelist=comm.index.values
    training_data=data.loc[comm.index] 
    nsamp=training_data.shape[1]
    n_neighbors=math.ceil(nsamp/10)
    n_neighbors
    print('Spectral pseudotime time')
    pt_spectral, latent_space_2d = estimate_pseudotime_using_spectral_embedding(training_data.T, n_neighbors)
    [fig, ax] = plot_latent_space_notrue(latent_space_2d)
    PseudoTimefile=f'{path_to_fig}LinearPseudotime{case}_{alpha}_comm_{commId}.pdf'

    pt = PdfPages(PseudoTimefile)
    fig.suptitle("Community="+str(commId) + ", "+ Flag + ', Neighbours=' +str(n_neighbors))
    fig
    pt.savefig(fig)

    pt.close()
    print('Gene plots')
    geneplot=genelist 
    idx=np.argsort(pt_spectral)
    figfile=f'{path_to_fig}PlotfilterSin{case}_{alpha}_comm_{commId}.pdf'
    pp = PdfPages(figfile)
    flag=0
    for g in geneplot:
        f=plt.figure()
        gene_series = data.loc[g][idx]
        #params,params_covariance=optimize.curve_fit(testsin_phase,pt_spectral[idx],gene_series,p0=[2,2,0])
        try:
            params, params_covariance = optimize.curve_fit(test_func, pt_spectral[idx],gene_series, p0=[1, 1, 2, 1])
            xtest = np.linspace(np.min(pt_spectral), np.max(pt_spectral), 100)
            ts=test_func(xtest,params[0],params[1],params[2],params[3])

            residuals=gene_series-test_func(pt_spectral[idx],params[0],params[1],params[2],params[3])

            rss=sum(residuals**2)
        
            #plt spline
            plt.plot(xtest, ts, color='m', lw=3);
            #plot data
            plt.scatter(pt_spectral[idx],gene_series)
            plt.title("Gene " + g + ", Com="+str(commId) + ", "+ Flag + ', RSS=' +str(rss) )
            pp.savefig(f)
        except RuntimeError:
            print("Error - curve_fit failed Gene " + g )
            file_obj.write("curve_fit failed Gene " + g )
            file_obj.write("\n")
            flag=1

    pp.close()
    if (flag==0):
        file_obj.write("no failed plots")
        file_obj.write("\n")
    
file_obj.close()

# Get data for the community of interest - Non Linear Case

In [ ]:
#Start from the linear communities
file_obj=open(filename,'a')
file_obj.write("NON LINEAR COMMUNITIES MISSING PLOTS\n ")
for commId in NonLinearId:
    print('Community=')
    print(commId)
    file_obj.write(f'Community ={commId}')
    file_obj.write("\n ")

    #commId=NonLinearId[12]
    Flag="Non Linear"
    comm = commData[commData.CommunityID == commId]
    #select ONLY the genes in the selected community to use laterin the plot
    genelist=comm.index.values
    training_data=data.loc[comm.index]
    nsamp=training_data.shape[1]
    n_neighbors=math.ceil(nsamp/10)
    n_neighbors
    pt_spectral, latent_space_2d = estimate_pseudotime_using_spectral_embedding(training_data.T, n_neighbors)
    [fig, ax] = plot_latent_space_notrue(latent_space_2d)
    PseudoTimefile=f'{path_to_fig}NonLinearPseudotime{case}_{alpha}_comm_{commId}.pdf'
    pt = PdfPages(PseudoTimefile)
    fig.suptitle("Community="+str(commId) + ", "+ Flag + ', Neighbours=' +str(n_neighbors))
    fig
    pt.savefig(fig)
    pt.close()
    #Plot genes
    geneplot=genelist
    idx=np.argsort(pt_spectral)
    figfile=f'{path_to_fig}Plotfilter{case}_{alpha}_comm_{commId}.pdf'
    pp = PdfPages(figfile)
    flag=0

    for g in geneplot:

        f=plt.figure()
        gene_series = data.loc[g][idx]
        try:
            params, params_covariance = optimize.curve_fit(test_func, pt_spectral[idx],gene_series, p0=[1, 1, 2, 1])
            xtest = np.linspace(np.min(pt_spectral), np.max(pt_spectral), 100)
            ts=test_func(xtest,params[0],params[1],params[2],params[3])

            residuals=gene_series-test_func(pt_spectral[idx],params[0],params[1],params[2],params[3])
            rss=sum(residuals**2)
        
            #plt spline
            plt.plot(xtest, ts, color='m', lw=3);
            #plot data
            plt.scatter(pt_spectral[idx],gene_series)
            plt.title("Gene " + g + ", Com="+str(commId) + ", "+ Flag + ', RSS=' +str(rss) )
            pp.savefig(f)        
        except RuntimeError:
            print("Error - curve_fit failed Gene " + g )
            file_obj.write("curve_fit failed Gene " + g )     
    pp.close()
    if (flag==0):
        file_obj.write("no failed plots")
        file_obj.write("\n")
file_obj.close()
